In [1]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_list=os.listdir('../../dataset/df_train.pkl.gz/')
df_train=pd.DataFrame()
for file in file_list:
    df_train=pd.concat([df_train, 
                        pd.read_pickle('../../dataset/df_train.pkl.gz/'+file, compression='gzip')])
df_train['mission_prgrm']=df_train['mission']+'; '+df_train['prgrm_dsc']
len(df_train)

229472

### Prepare parrallel envionment.

In [3]:
import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]


In [8]:
dview.execute('from sklearn import model_selection, preprocessing, naive_bayes, metrics')
dview.execute('from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer')
dview.execute('from sklearn import decomposition, ensemble')
dview.execute('from nltk.stem import PorterStemmer')
dview.execute('from nltk import word_tokenize')
dview.execute('from nltk.stem import WordNetLemmatizer')
dview.execute('from nltk.corpus import wordnet')
dview.execute('import pandas as pd')
dview.execute('import nltk')
dview['df_train']=df_train
dview['df_performance']=pd.DataFrame(columns=['trial', 'txt_field', 'classifier', 'tokenizer', 'vect_type', 'average_mtd', 'accuracy', 'precision', 'recall', 'f1'])

In [9]:
@dview.parallel(block=True)
def func_naive_bayes(trial):
    global df_train, df_performance, txt_field, classifier, tokenizer, vect_type, average_mtd

    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################

    def porter_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]
    
    # Lemmatize using POS tags, assume to improve accuracy.
    # Ref: 
    #   - https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
    #   - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    def lemma_tokenizer(str_input):
        tokens=word_tokenize(str_input)
        return [WordNetLemmatizer().lemmatize(word=word, pos=get_wordnet_pos(pos)) for word, pos in nltk.pos_tag(tokens)]
            
    if tokenizer=='lemma':
        tokenizer=lemma_tokenizer
    elif tokenizer=='porter':
        tokenizer=porter_tokenizer
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':tokenizer.__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(y_pred=predictions, y_true=y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(y_pred=predictions, y_true=y_valid, average=average_mtd),
                                            'f1':metrics.f1_score(y_pred=predictions, y_true=y_valid, average=average_mtd)
                                           }, ignore_index=True)

### Iterate different configurations at once.

In [ ]:
for txt_field in ['mission', 'prgrm_dsc', 'mission_prgrm']:
    for classifier in [naive_bayes.MultinomialNB(), naive_bayes.ComplementNB()]:
        for tokenizer in ['lemma', 'porter']:
            for vect_type in ['count', 'tfidf']:
                for average_mtd in ['macro', 'weighted']:
                    dview['txt_field']=txt_field
                    dview['classifier']=classifier
                    dview['tokenizer']=tokenizer
                    dview['vect_type']=vect_type
                    dview['average_mtd']=average_mtd
                    t=func_naive_bayes.map(range(48))

In [ ]:
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

In [14]:
df_performance[df_performance.average_mtd=='macro'].groupby(['txt_field', 'classifier', 'tokenizer', 'vect_type']).describe()[['accuracy','f1']]

accuracy  \
                                                                                             count   
txt_field     classifier                                         tokenizer      vect_type            
mission       ComplementNB(alpha=1.0, class_prior=None, fit_p... LemmaTokenizer count         48.0   
                                                                                tfidf         48.0   
                                                                 function       count         48.0   
                                                                                tfidf         48.0   
              MultinomialNB(alpha=1.0, class_prior=None, fit_... LemmaTokenizer count         48.0   
                                                                                tfidf         48.0   
                                                                 function       count         48.0   
                                                                                tfidf         48.0   
mission_prgrm ComplementNB(alpha=1.0, class_prior=None, fit_p... LemmaTokenizer count         48.0   
                                                                                tfidf         48.0   
                                                                 function       count         48.0   
                                                                                tfidf         48.0   
              MultinomialNB(alpha=1.0, class_prior=None, fit_... LemmaTokenizer count         48.0   
                                                                                tfidf         48.0   
                                                                 function       count         48.0   
                                                                                tfidf         48.0   
prgrm_dsc     ComplementNB(alpha=1.0, class_prior=None, fit_p... LemmaTokenizer count         48.0   
                                                                                tfidf         48.0   
                                                                 function       count         48.0   
                                                                                tfidf         48.0   
              MultinomialNB(alpha=1.0, class_prior=None, fit_... LemmaTokenizer count         48.0   
                                                                                tfidf         48.0   
                                                                 function       count         48.0   
                                                                                tfidf         48.0   

                                                                                                     \
                                                                                               mean   
txt_field     classifier                                         tokenizer      vect_type             
mission       ComplementNB(alpha=1.0, class_prior=None, fit_p... LemmaTokenizer count      0.699960   
                                                                                tfidf      0.697922   
                                                                 function       count      0.695816   
                                                                                tfidf      0.693615   
              MultinomialNB(alpha=1.0, class_prior=None, fit_... LemmaTokenizer count      0.661500   
                                                                                tfidf      0.559326   
                                                                 function       count      0.660228   
                                                                                tfidf      0.557107   
mission_prgrm ComplementNB(alpha=1.0, class_prior=None, fit_p... LemmaTokenizer count      0.706067   
                                                                                tfidf      0.713537

## Draft.

### Try Mission Statements - MultinomialNB - LemmaTokenizer - TFIDF.

In [29]:
@dview.parallel(block=True)
def func_mission_MNB_lemma_tfidf(trial):
    global df_train, df_performance, txt_field, classifier, tokenizer, vect_type, average_mtd
    
    ##########################################################
    ####### Set environments for different functions #########
    txt_field='mission' # 'mission', 'prgrm_dsc', 'mission_prgrm'
    classifier=naive_bayes.MultinomialNB()
    tokenizer='lemma' # 'lemma', 'stemming'
    vect_type='tfidf' # 'count', 'tfidf'
    average_mtd='macro' # Use unweighted mean.
    ####### Set environments for different functions #########
    ##########################################################
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
            
    if tokenizer=='lemma':
        tokenizer=LemmaTokenizer()
    elif tokenizer=='stemming':
        tokenizer=stemming_tokenizer()
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':type(tokenizer).__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(predictions, y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(predictions, y_valid, average=average_mtd),
                                           }, ignore_index=True)

In [30]:
t=func_mission_MNB_lemma_tfidf.map(range(100))
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

,trial,txt_field,classifier,tokenizer,vect_type,average_mtd,accuracy,precision,recall
681,97,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,tfidf,macro,0.673533,0.479451,0.639056
353,51,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.595300,0.353232,0.595305
595,85,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.703933,0.533654,0.635387
561,81,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.662233,0.426434,0.689126
240,34,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.598700,0.355358,0.670974
347,49,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,tfidf,macro,0.700633,0.540676,0.633220
662,94,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.656733,0.475081,0.608428
79,10,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,tfidf,macro,0.699167,0.529788,0.612299
335,47,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,tfidf,macro,0.561700,0.300220,0.537231
269,39,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.609900,0.363073,0.620524


### Try Mission Statements - MultinomialNB - LemmaTokenizer - Count.

In [5]:
@dview.parallel(block=True)
def func_mission_MNB_lemma_count(trial):
    global df_train, df_performance
    
    ##########################################################
    ####### Set environments for different functions #########
    txt_field='mission' # 'mission', 'prgrm_dsc', 'mission_prgrm'
    classifier=naive_bayes.MultinomialNB()
    tokenizer='lemma' # 'lemma', 'stemming'
    vect_type='count' # 'count', 'tfidf'
    average_mtd='macro' # Use unweighted mean.
    ####### Set environments for different functions #########
    ##########################################################
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
            
    if tokenizer=='lemma':
        tokenizer=LemmaTokenizer()
    elif tokenizer=='stemming':
        tokenizer=stemming_tokenizer()
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':type(tokenizer).__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(predictions, y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(predictions, y_valid, average=average_mtd),
                                           }, ignore_index=True)

In [6]:
t=func_mission_MNB_lemma_count.map(range(100))
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

### Try Program Description - MultinomialNB - LemmaTokenizer - Count.

In [12]:
@dview.parallel(block=True)
def func_prgrm_MNB_lemma_count(trial):
    global df_train, df_performance
    
    ##########################################################
    ####### Set environments for different functions #########
    txt_field='prgrm_dsc' # 'mission', 'prgrm_dsc', 'mission_prgrm'
    classifier=naive_bayes.MultinomialNB()
    tokenizer='lemma' # 'lemma', 'stemming'
    vect_type='count' # 'count', 'tfidf'
    average_mtd='macro' # Use unweighted mean.
    ####### Set environments for different functions #########
    ##########################################################
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
            
    if tokenizer=='lemma':
        tokenizer=LemmaTokenizer()
    elif tokenizer=='stemming':
        tokenizer=stemming_tokenizer()
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':type(tokenizer).__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(predictions, y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(predictions, y_valid, average=average_mtd),
                                           }, ignore_index=True)

In [14]:
t=func_prgrm_MNB_lemma_count.map(range(100))
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

,trial,txt_field,classifier,tokenizer,vect_type,average_mtd,accuracy,precision,recall
4,1,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.598833,0.349254,0.579754
13,7,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.658333,0.422397,0.644755
52,26,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.656933,0.420701,0.683195
176,88,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.662600,0.426009,0.652042
56,28,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.667333,0.431431,0.671606
167,83,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.600867,0.357212,0.680094
11,5,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.604600,0.355520,0.627683
194,96,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.603167,0.353313,0.623195
28,14,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.660667,0.426780,0.647043
188,94,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.662300,0.423771,0.637518


### Try Mission Statements - ComplementNB - LemmaTokenizer - Count.

In [18]:
@dview.parallel(block=True)
def func_mission_CNB_lemma_count(trial):
    global df_train, df_performance
    
    ##########################################################
    ####### Set environments for different functions #########
    txt_field='mission' # 'mission', 'prgrm_dsc', 'mission_prgrm'
    classifier=naive_bayes.ComplementNB()
    tokenizer='lemma' # 'lemma', 'stemming'
    vect_type='count' # 'count', 'tfidf'
    average_mtd='macro' # Use unweighted mean.
    ####### Set environments for different functions #########
    ##########################################################
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
            
    if tokenizer=='lemma':
        tokenizer=LemmaTokenizer()
    elif tokenizer=='stemming':
        tokenizer=stemming_tokenizer()
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':type(tokenizer).__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(predictions, y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(predictions, y_valid, average=average_mtd),
                                           }, ignore_index=True)

In [19]:
t=func_mission_CNB_lemma_count.map(range(100))
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

,trial,txt_field,classifier,tokenizer,vect_type,average_mtd,accuracy,precision,recall
115,29,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.606267,0.356488,0.618608
209,53,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.663600,0.426699,0.640428
146,36,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.606667,0.353021,0.649588
126,30,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.699667,0.534671,0.630701
155,39,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.609900,0.363073,0.620524
203,51,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.595300,0.353232,0.595305
215,53,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.700400,0.534318,0.625981
15,3,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.601300,0.353904,0.650824
17,5,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.604600,0.355520,0.627683
240,60,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.663200,0.428999,0.678820


### Try Program Description - ComplementNB - LemmaTokenizer - Count.

In [16]:
@dview.parallel(block=True)
def func_prgrm_CNB_lemma_count(trial):
    global df_train, df_performance
    
    ##########################################################
    ####### Set environments for different functions #########
    txt_field='prgrm_dsc' # 'mission', 'prgrm_dsc', 'mission_prgrm'
    classifier=naive_bayes.ComplementNB()
    tokenizer='lemma' # 'lemma', 'stemming'
    vect_type='count' # 'count', 'tfidf'
    average_mtd='macro' # Use unweighted mean.
    ####### Set environments for different functions #########
    ##########################################################
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
            
    if tokenizer=='lemma':
        tokenizer=LemmaTokenizer()
    elif tokenizer=='stemming':
        tokenizer=stemming_tokenizer()
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':type(tokenizer).__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(predictions, y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(predictions, y_valid, average=average_mtd),
                                           }, ignore_index=True)

In [17]:
t=func_prgrm_CNB_lemma_count.map(range(100))
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

,trial,txt_field,classifier,tokenizer,vect_type,average_mtd,accuracy,precision,recall
3,0,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.601267,0.351739,0.625842
201,67,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.598733,0.353515,0.625180
289,97,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.657133,0.422356,0.644037
284,94,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.598733,0.352534,0.630569
109,37,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.661867,0.428198,0.653606
248,82,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.600867,0.351543,0.663272
240,80,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.659667,0.426818,0.636164
190,62,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.657400,0.478270,0.614129
106,34,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.655733,0.479513,0.613932
204,68,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.659033,0.424482,0.680102


### Try Program Description - ComplementNB - LemmaTokenizer - TFIDF.

In [23]:
@dview.parallel(block=True)
def func_prgrm_CNB_lemma_tfidf(trial):
    global df_train, df_performance
    
    ##########################################################
    ####### Set environments for different functions #########
    txt_field='prgrm_dsc' # 'mission', 'prgrm_dsc', 'mission_prgrm'
    classifier=naive_bayes.ComplementNB()
    tokenizer='lemma' # 'lemma', 'stemming'
    vect_type='tfidf' # 'count', 'tfidf'
    average_mtd='macro' # Use unweighted mean.
    ####### Set environments for different functions #########
    ##########################################################
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
            
    if tokenizer=='lemma':
        tokenizer=LemmaTokenizer()
    elif tokenizer=='stemming':
        tokenizer=stemming_tokenizer()
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':type(tokenizer).__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(predictions, y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(predictions, y_valid, average=average_mtd),
                                           }, ignore_index=True)

In [24]:
t=func_prgrm_CNB_lemma_tfidf.map(range(100))
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

,trial,txt_field,classifier,tokenizer,vect_type,average_mtd,accuracy,precision,recall
96,18,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.700400,0.528067,0.616052
251,51,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.661267,0.426321,0.683731
236,46,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.701200,0.531807,0.626519
284,56,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.653367,0.477555,0.613562
234,46,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.655467,0.474524,0.621911
405,81,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.660900,0.484696,0.616098
365,73,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.656233,0.477075,0.610757
461,93,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.657600,0.423429,0.641899
47,11,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.661233,0.424397,0.657256
18,3,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.601300,0.353904,0.650824


### Try Mission Statements - ComplementNB - LemmaTokenizer - TFIDF.

In [25]:
@dview.parallel(block=True)
def func_mission_CNB_lemma_tfidf(trial):
    global df_train, df_performance
    
    ##########################################################
    ####### Set environments for different functions #########
    txt_field='mission' # 'mission', 'prgrm_dsc', 'mission_prgrm'
    classifier=naive_bayes.ComplementNB()
    tokenizer='lemma' # 'lemma', 'stemming'
    vect_type='tfidf' # 'count', 'tfidf'
    average_mtd='macro' # Use unweighted mean.
    ####### Set environments for different functions #########
    ##########################################################
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF[txt_field].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
            
    if tokenizer=='lemma':
        tokenizer=LemmaTokenizer()
    elif tokenizer=='stemming':
        tokenizer=stemming_tokenizer()
    ################ Define tokenizer ################
    ##########################################################
    
    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.
    
    if vect_type=='count':
        ##### Token counts #####
        # create the transform
        vectorizer = CountVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)
    elif vect_type=='tfidf':
        ##### TF-IDF #####
        # create the transform
        vectorizer = TfidfVectorizer(stop_words='english', 
                                     tokenizer=tokenizer, 
                                     analyzer='word'
                                    )
        # tokenize and build vocab
        vectorizer.fit(trainDF['text'])
        # Encode document: transform the training and validation data using count vectorizer object
        x_train_vect =  vectorizer.transform(x_train)
        x_valid_vect =  vectorizer.transform(x_valid)

    ######### Text Vectorization and Transformation ##########
    ##########################################################
    
    classifier.fit(x_train_vect, y_train)
    predictions = classifier.predict(x_valid_vect)
    df_performance = df_performance.append({'trial':str(trial), 
                                            'txt_field':txt_field, 
                                            'classifier':str(classifier), 
                                            'tokenizer':type(tokenizer).__name__, 
                                            'vect_type':vect_type, 
                                            'average_mtd':average_mtd,
                                            'accuracy':metrics.accuracy_score(predictions, y_valid), 
                                            'precision':metrics.precision_score(predictions, y_valid, average=average_mtd),
                                            'recall':metrics.recall_score(predictions, y_valid, average=average_mtd),
                                           }, ignore_index=True)

In [26]:
t=func_mission_CNB_lemma_tfidf.map(range(100))
df_performance=pd.concat(dview.gather('df_performance'), ignore_index=True)
df_performance.sample(10)

,trial,txt_field,classifier,tokenizer,vect_type,average_mtd,accuracy,precision,recall
248,40,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,tfidf,macro,0.673467,0.481593,0.663747
411,69,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.603033,0.353719,0.629368
51,6,mission,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,tfidf,macro,0.693267,0.533007,0.631130
219,37,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.608833,0.358827,0.599976
373,63,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.665100,0.430047,0.652673
530,88,prgrm_dsc,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.604800,0.353762,0.596581
96,16,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.659567,0.425299,0.648161
377,63,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.655667,0.474790,0.606161
193,33,mission,"MultinomialNB(alpha=1.0, class_prior=None, fit...",LemmaTokenizer,count,macro,0.657933,0.423631,0.641981
88,14,prgrm_dsc,"ComplementNB(alpha=1.0, class_prior=None, fit_...",LemmaTokenizer,count,macro,0.662267,0.481588,0.611395


In [28]:
df_performance.groupby(['txt_field', 'classifier', 'vect_type']).describe()

accuracy  \
                                                                          count   
txt_field classifier                                         vect_type            
mission   ComplementNB(alpha=1.0, class_prior=None, fit_p... count        100.0   
                                                             tfidf        100.0   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count        100.0   
prgrm_dsc ComplementNB(alpha=1.0, class_prior=None, fit_p... count        100.0   
                                                             tfidf        100.0   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count        100.0   

                                                                                  \
                                                                            mean   
txt_field classifier                                         vect_type             
mission   ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.699907   
                                                             tfidf      0.697841   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.661342   
prgrm_dsc ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.657213   
                                                             tfidf      0.673393   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.602443   

                                                                                  \
                                                                             std   
txt_field classifier                                         vect_type             
mission   ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.002527   
                                                             tfidf      0.002500   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.002978   
prgrm_dsc ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.002682   
                                                             tfidf      0.002592   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.002772   

                                                                                  \
                                                                             min   
txt_field classifier                                         vect_type             
mission   ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.693900   
                                                             tfidf      0.690767   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.651700   
prgrm_dsc ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.650667   
                                                             tfidf      0.667533   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.595300   

                                                                                  \
                                                                             25%   
txt_field classifier                                         vect_type             
mission   ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.698317   
                                                             tfidf      0.695900   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.659417   
prgrm_dsc ComplementNB(alpha=1.0, class_prior=None, fit_p... count      0.655233   
                                                             tfidf      0.671317   
          MultinomialNB(alpha=1.0, class_prior=None, fit_... count      0.600658   

                                                                                  \
                                                                             50%   
txt_field classifier                                         vect_type             
mission   ComplementNB(alpha=1.0, class_pri

In [5]:
df_mission_MNB_lemma_count.describe()

,accuracy,precision,recall
count,100.000000,100.000000,100.000000
mean,0.661060,0.425503,0.658984
std,0.002628,0.002539,0.020294
min,0.655533,0.419339,0.627483
25%,0.659017,0.423844,0.644080
50%,0.661350,0.425360,0.650408
75%,0.662775,0.427471,0.677598
max,0.668267,0.432884,0.705346


### Try Mission Statements - ComplementNB - LemmaTokenizer - Count.

In [ ]:
df_mission_CNB_lemma_count=pd.DataFrame(columns=['accuracy', 'precision', 'recall'])
for trial in tqdm(range(0, 100)):
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF['mission'].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    ################ Define tokenizer ################
    ##########################################################

    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.

    ##### Token counts #####
    # create the transform
    count_vect = CountVectorizer(stop_words='english', 
                                 tokenizer=LemmaTokenizer(), 
                                 analyzer='word'
                                )
    # tokenize and build vocab
    count_vect.fit(trainDF['text'])
    # Encode document: transform the training and validation data using count vectorizer object
    x_train_vect_count =  count_vect.transform(x_train)
    x_valid_vect_count =  count_vect.transform(x_valid)
    ##### Token counts #####

    ##### TF-IDF #####
    # create the transform
    tfidf_vect = TfidfVectorizer(stop_words='english', 
                                 tokenizer=LemmaTokenizer(), 
                                 analyzer='word'
                                )
    # tokenize and build vocab
    tfidf_vect.fit(trainDF['text'])
    # Encode document: transform the training and validation data using count vectorizer object
    x_train_vect_tfidf =  tfidf_vect.transform(x_train)
    x_valid_vect_tfidf =  tfidf_vect.transform(x_valid)
    ##### TF-IDF #####

    ######### Text Vectorization and Transformation ##########
    ##########################################################

    def func_performance(classifier, x_train, y_train, x_valid, y_valid):
        # fit the training dataset on the classifier
        classifier.fit(x_train, y_train)
        # predict the labels on validation dataset
        predictions = classifier.predict(x_valid)
        return [metrics.accuracy_score(predictions, y_valid), 
                metrics.precision_score(predictions, y_valid, 
                                        average='macro', # Use unweighted mean.
                                       ),
                metrics.recall_score(predictions, y_valid, 
                                     average='macro',  # Use unweighted mean.
                                    )]

    performance_result=func_performance(classifier=naive_bayes.ComplementNB(), 
                                        x_train=x_train_vect_count,
                                        y_train= y_train, 
                                        x_valid=x_valid_vect_count,
                                        y_valid=y_valid
                                       )
    df_mission_CNB_lemma_count.loc[trial]=performance_result

 85%|████████▌ | 85/100 [3:10:23<33:50, 135.39s/it]

In [ ]:
df_mission_CNB_lemma_count.describe()

### Try Program Description - MultinomialNB - LemmaTokenizer - Count.

In [7]:
df_prgrm_MNB_lemma_count=pd.DataFrame(columns=['accuracy', 'precision', 'recall'])
for trial in tqdm(range(0, 100)):
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF['prgrm_dsc'].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    ################ Define tokenizer ################
    ##########################################################

    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.

    ##### Token counts #####
    # create the transform
    count_vect = CountVectorizer(stop_words='english', 
                                 tokenizer=LemmaTokenizer(), 
                                 analyzer='word'
                                )
    # tokenize and build vocab
    count_vect.fit(trainDF['text'])
    # Encode document: transform the training and validation data using count vectorizer object
    x_train_vect_count =  count_vect.transform(x_train)
    x_valid_vect_count =  count_vect.transform(x_valid)
    ##### Token counts #####

    ##### TF-IDF #####
    # create the transform
    tfidf_vect = TfidfVectorizer(stop_words='english', 
                                 tokenizer=LemmaTokenizer(), 
                                 analyzer='word'
                                )
    # tokenize and build vocab
    tfidf_vect.fit(trainDF['text'])
    # Encode document: transform the training and validation data using count vectorizer object
    x_train_vect_tfidf =  tfidf_vect.transform(x_train)
    x_valid_vect_tfidf =  tfidf_vect.transform(x_valid)
    ##### TF-IDF #####

    ######### Text Vectorization and Transformation ##########
    ##########################################################

    def func_performance(classifier, x_train, y_train, x_valid, y_valid):
        # fit the training dataset on the classifier
        classifier.fit(x_train, y_train)
        # predict the labels on validation dataset
        predictions = classifier.predict(x_valid)
        return [metrics.accuracy_score(predictions, y_valid), 
                metrics.precision_score(predictions, y_valid, 
                                        average='macro', # Use unweighted mean.
                                       ),
                metrics.recall_score(predictions, y_valid, 
                                     average='macro',  # Use unweighted mean.
                                    )]

    performance_result=func_performance(classifier=naive_bayes.MultinomialNB(), 
                                        x_train=x_train_vect_count,
                                        y_train= y_train, 
                                        x_valid=x_valid_vect_count,
                                        y_valid=y_valid
                                       )
    df_prgrm_MNB_lemma_count.loc[trial]=performance_result

100%|██████████| 100/100 [11:35:52<00:00, 419.77s/it]


In [ ]:
df_prgrm_MNB_lemma_count.describe()

### Try Program Description - ComplementNB - LemmaTokenizer - Count.

In [ ]:
df_prgrm_CNB_lemma_count=pd.DataFrame(columns=['accuracy', 'precision', 'recall'])
for trial in tqdm(range(0, 100)):
    
    ##########################################################
    ################ Prepare dataframe for ML ################
    #### Sample ####
    small_num=0
    while small_num<100: # Make sure each category has at least 100 records.
        trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(100000)
        small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    #### Sample ####
    trainDF['text'] = trainDF['prgrm_dsc'].astype(str)
    trainDF['label'] = trainDF['NTEE1'].astype(str)
    # split the dataset into training and validation datasets 
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(trainDF['text'], trainDF['label'],
                                                                          train_size=0.7, shuffle=True)
    ################ Prepare dataframe for ML ################
    ##########################################################

    ##########################################################
    ################ Define tokenizer ################
    # Source: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/
    # Use NLTK's PorterStemmer
    def stemming_tokenizer(str_input):
        tokens = word_tokenize(str_input)
        return [PorterStemmer().stem(token) for token in tokens]

    # Source: https://scikit-learn.org/stable/modules/feature_extraction.html
    # Use NLTK's Lemmatizer
    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
             return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    ################ Define tokenizer ################
    ##########################################################

    ##########################################################
    ######### Text Vectorization and Transformation ##########
    # 1. Use Porter Stemmer.
    # 2. Use word level, character level does not make sense for current situation.
    # 3. Use count (freq) and tf-idf vectorizer. see: 
    # Bengfort, B., Bilbro, R., & Ojeda, T. (2018). Applied Text Analysis with Python: Enabling Language-Aware Data Products with Machine Learning (1 edition). Beijing Boston Farnham Sebastopol Tokyo: O’Reilly Media.
    # Page: 67.

    ##### Token counts #####
    # create the transform
    count_vect = CountVectorizer(stop_words='english', 
                                 tokenizer=LemmaTokenizer(), 
                                 analyzer='word'
                                )
    # tokenize and build vocab
    count_vect.fit(trainDF['text'])
    # Encode document: transform the training and validation data using count vectorizer object
    x_train_vect_count =  count_vect.transform(x_train)
    x_valid_vect_count =  count_vect.transform(x_valid)
    ##### Token counts #####

    ##### TF-IDF #####
    # create the transform
    tfidf_vect = TfidfVectorizer(stop_words='english', 
                                 tokenizer=LemmaTokenizer(), 
                                 analyzer='word'
                                )
    # tokenize and build vocab
    tfidf_vect.fit(trainDF['text'])
    # Encode document: transform the training and validation data using count vectorizer object
    x_train_vect_tfidf =  tfidf_vect.transform(x_train)
    x_valid_vect_tfidf =  tfidf_vect.transform(x_valid)
    ##### TF-IDF #####

    ######### Text Vectorization and Transformation ##########
    ##########################################################

    def func_performance(classifier, x_train, y_train, x_valid, y_valid):
        # fit the training dataset on the classifier
        classifier.fit(x_train, y_train)
        # predict the labels on validation dataset
        predictions = classifier.predict(x_valid)
        return [metrics.accuracy_score(predictions, y_valid), 
                metrics.precision_score(predictions, y_valid, 
                                        average='macro', # Use unweighted mean.
                                       ),
                metrics.recall_score(predictions, y_valid, 
                                     average='macro',  # Use unweighted mean.
                                    )]

    performance_result=func_performance(classifier=naive_bayes.ComplementNB(), 
                                        x_train=x_train_vect_count,
                                        y_train= y_train, 
                                        x_valid=x_valid_vect_count,
                                        y_valid=y_valid
                                       )
    df_prgrm_CNB_lemma_count.loc[trial]=performance_result

In [ ]:
df_prgrm_CNB_lemma_count.describe()

In [27]:
# Naive Bayes on Word Level TF IDF Vectors
# Naive Bayes on Count Vectors
accuracy = train_model(classifier=naive_bayes.MultinomialNB(), 
                       x_train=x_train_vect_tfidf,
                       y_train= y_train, 
                       x_valid=x_valid_vect_tfidf,
                       y_valid=y_valid
                      )
results.loc[len(results)] = ["NB, Count Vectors", accuracy[0], accuracy[1], accuracy[2], accuracy[3]]
print("NB, TF IDF Vectors: ", accuracy)


# Raw: MNB, Count Vectors:  [0.6732666666666667, 0.4503852200520778, 0.6661678975327818, datetime.timedelta(microseconds=489443)]
# Raw: CNB, Count Vectors:  [0.7013333333333334, 0.5275971009306734, 0.6170202787685387, datetime.timedelta(microseconds=521657)]
# Stemmed: MNB, Count Vectors:  [0.6601, 0.4243013182900721, 0.639930088219624, datetime.timedelta(microseconds=443571)]
# Stemmed: CNB, Count Vectors:  [0.7, 0.5320034132229355, 0.6383184621220114, datetime.timedelta(microseconds=504788)]
# Lemma: MNB, Count Vectors:  [0.6615, 0.42366552445612, 0.6783346552845068, datetime.timedelta(microseconds=572416)]
# Lemma: CNB, Count Vectors:  [0.7021333333333334, 0.5341465196920103, 0.6318440519847688, datetime.timedelta(microseconds=586597)]

# Raw: MNB, TF IDF Vectors:  [0.5859, 0.32327746269021723, 0.5485416630089535, datetime.timedelta(microseconds=536424)]
# Raw: CNB, TF IDF Vectors:  [0.6992333333333334, 0.5306246321804787, 0.6136847132057796, datetime.timedelta(microseconds=580513)]
# Stemmed: MNB, TF IDF Vectors:  [0.553, 0.2949552239378839, 0.536824161513825, datetime.timedelta(microseconds=453431)]
# Stemmed: CNB, TF IDF Vectors:  [0.6967666666666666, 0.5349583083797411, 0.6223488506468897, datetime.timedelta(microseconds=520307)]
# Lemma: MNB, TF IDF Vectors:  [0.5589666666666666, 0.297005157029138, 0.5293106710625075, datetime.timedelta(microseconds=529497)]
# Lemma: CNB, TF IDF Vectors:  [0.7012333333333334, 0.536867845262306, 0.6273414839488708, datetime.timedelta(microseconds=546401)]

NB, TF IDF Vectors:  [0.5583666666666667, 0.29982304712090974, 0.5961215483457072, datetime.timedelta(microseconds=498992)]


/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


**Looks like `Lemma-CNB-Count` produces best results.**